In [9]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load data
data_path = 'IP_decisions_2/IP_decisions_tucson_noise=0.01_model=1200'
data = pd.read_csv(data_path+'.csv')

print(data.head())

   Unnamed: 0  time  Site Outdoor Air Drybulb Temperature(Environment)  \
0           0  8.75                                          -6.010972   
1           1  8.75                                          -6.010972   
2           2  8.75                                          -6.010972   
3           3  8.75                                          -6.010972   
4           4  8.75                                          -6.010972   

   Site Outdoor Air Relative Humidity(Environment)  \
0                                        57.108601   
1                                        57.108601   
2                                        57.108601   
3                                        57.108601   
4                                        57.108601   

   Site Wind Speed(Environment)  \
0                      5.521787   
1                      5.521787   
2                      5.521787   
3                      5.521787   
4                      5.521787   

   Site Direct Sola

In [10]:
''' categorize data into unique tuples of ['time', 
'Site Outdoor Air Drybulb Temperature(Environment)',
'Site Outdoor Air Relative Humidity(Environment)',
'Site Wind Speed(Environment)',
'Site Direct Solar Radiation Rate per Area(Environment)',
'Zone People Occupant Count(SPACE1-1)',
'Zone Air Temperature(SPACE1-1)']
'''

categorized_data = pd.DataFrame(columns=['time',
                                'Site Outdoor Air Drybulb Temperature(Environment)',
                                'Site Outdoor Air Relative Humidity(Environment)',
                                'Site Wind Speed(Environment)',
                                'Site Direct Solar Radiation Rate per Area(Environment)',
                                'Zone People Occupant Count(SPACE1-1)',
                                'Zone Air Temperature(SPACE1-1)',
                                'actions'])

current_flag = 0
actions = []
for row_index in range(len(data)):
    row = data.iloc[row_index]
    a = row['action']
    actions.append(a)
    current_flag += 1
    if current_flag == 30:
        time = row['time']
        temp = row['Site Outdoor Air Drybulb Temperature(Environment)']
        hum = row['Site Outdoor Air Relative Humidity(Environment)']
        wind = row['Site Wind Speed(Environment)']
        solar = row['Site Direct Solar Radiation Rate per Area(Environment)']
        people = row['Zone People Occupant Count(SPACE1-1)']
        zone_temp = row['Zone Air Temperature(SPACE1-1)']
        # categorized_data = categorized_data.append({'time': time,
        #                                             'Site Outdoor Air Drybulb Temperature(Environment)': temp,
        #                                             'Site Outdoor Air Relative Humidity(Environment)': hum,
        #                                             'Site Wind Speed(Environment)': wind,
        #                                             'Site Direct Solar Radiation Rate per Area(Environment)': solar,
        #                                             'Zone People Occupant Count(SPACE1-1)': people,
        #                                             'Zone Air Temperature(SPACE1-1)': zone_temp,
        #                                             'actions': actions}, ignore_index=True)
        # do the above using concat
        df = pd.DataFrame({'time': [time],
                            'Site Outdoor Air Drybulb Temperature(Environment)': [temp],
                            'Site Outdoor Air Relative Humidity(Environment)': [hum],
                            'Site Wind Speed(Environment)': [wind],
                            'Site Direct Solar Radiation Rate per Area(Environment)': [solar],
                            'Zone People Occupant Count(SPACE1-1)': [people],
                            'Zone Air Temperature(SPACE1-1)': [zone_temp],
                            'actions': [actions]})
        categorized_data = pd.concat([categorized_data, df], ignore_index=True)
        current_flag = 0
        actions = []

# write to csv
categorized_data.to_csv(data_path+'_categorized.csv', index=False)

In [11]:
# for every actions in new data, find the most frequent action
actions = categorized_data['actions']
actions = actions.tolist()
actions = np.array(actions)

policy_data = pd.DataFrame(columns=['time',
                                'Site Outdoor Air Drybulb Temperature(Environment)',
                                'Site Outdoor Air Relative Humidity(Environment)',
                                'Site Wind Speed(Environment)',
                                'Site Direct Solar Radiation Rate per Area(Environment)',
                                'Zone People Occupant Count(SPACE1-1)',
                                'Zone Air Temperature(SPACE1-1)',
                                'action'])

method = 'mode'
# method = 'mean'

for row_index in range(len(categorized_data)):
    row = categorized_data.iloc[row_index]
    time = row['time']
    temp = row['Site Outdoor Air Drybulb Temperature(Environment)']
    hum = row['Site Outdoor Air Relative Humidity(Environment)']
    wind = row['Site Wind Speed(Environment)']
    solar = row['Site Direct Solar Radiation Rate per Area(Environment)']
    people = row['Zone People Occupant Count(SPACE1-1)']
    zone_temp = row['Zone Air Temperature(SPACE1-1)']

    if method == 'mode':
        # find the mode of actions[row_index]
        counts = {}
        for a in actions[row_index]:
            if a in counts:
                counts[a] += 1
            else:
                counts[a] = 1
        action = max(counts, key=counts.get)
    elif method == 'mean':
        action = np.mean(actions[row_index])
        # round to nearest integer
        action = int(round(action))
    # policy_data = policy_data.append({'time': time,
    #                                   'Site Outdoor Air Drybulb Temperature(Environment)': temp,
    #                                   'Site Outdoor Air Relative Humidity(Environment)': hum,
    #                                   'Site Wind Speed(Environment)': wind,
    #                                   'Site Direct Solar Radiation Rate per Area(Environment)': solar,
    #                                   'Zone People Occupant Count(SPACE1-1)': people,
    #                                   'Zone Air Temperature(SPACE1-1)': zone_temp,
    #                                   'action': action}, ignore_index=True)
    # do the above using concat
    df = pd.DataFrame({'time': [time],
                        'Site Outdoor Air Drybulb Temperature(Environment)': [temp],
                        'Site Outdoor Air Relative Humidity(Environment)': [hum],
                        'Site Wind Speed(Environment)': [wind],
                        'Site Direct Solar Radiation Rate per Area(Environment)': [solar],
                        'Zone People Occupant Count(SPACE1-1)': [people],
                        'Zone Air Temperature(SPACE1-1)': [zone_temp],
                        'action': [action]})
    policy_data = pd.concat([policy_data, df], ignore_index=True)

# write to csv
policy_data.to_csv(data_path+'_policy.csv', index=False)